# Tutorial 4. Components

:::{note}
:icon: false

#### How to think clearly about panes, widgets, layouts, and `panel_material_ui.Page`

Most Panel confusion comes from mixing up **what shows data**, **what captures input**, and **what arranges the page**.

This tutorial gives you a compact mental model so you can choose the right component quickly.

## Learning goals

After this tutorial, you should be able to:

1. Explain the difference between panes, widgets, and layouts.
2. Use `pn.panel()` intentionally to wrap Python objects.
3. Build a small app with `pn.Column`/`pn.Row`.
4. Replace old template thinking with `panel_material_ui.Page`.
:::

---

In [ ]:
import panel as pn

pn.extension(notifications=True)


## 1) One mental model

Think in layers:

1. **Pane** = *render output* (text, dataframe, plot, image, markdown, etc.)
2. **Widget** = *collect input* (`value` comes from the user and/or Python)
3. **Layout** = *arrange components* (`Row`, `Column`, `Tabs`, `GridSpec`, ...)
4. **Page** (`panel_material_ui.Page`) = *app shell* (header/sidebar/main + theme system)

If you remember only one thing: **widgets ask, panes show, layouts organize, pages frame the app**.

All the Panel components can be visualized on the [Component Gallery](https://panel.holoviz.org/reference/index.html).

:::{exercise}

Visit the *Component Gallery* and spend a few minutes exploring the components it exposes. Spoiler alert, working on a Panel app you will spend a lot of time on this page looking for the right components and learning how they work!
:::

:::{tip}

Components usually have in their docstring a link to their documentation page, use `<component>?` in a notebook or your IDE inspection capabilities to access the link.
:::

---

## 2) Panes: display objects

Panes are wrappers around "things you want to show".

In [ ]:
import pandas as pd
import panel as pn

pn.extension()

markdown = pn.pane.Markdown("## Revenue report")
table = pn.pane.DataFrame(pd.DataFrame({"region": ["NA", "EU"], "sales": [12, 9]}), width=280)

pn.Column(markdown, table)

### `pn.panel()` convenience

`pn.panel(obj)` picks an appropriate pane for an object automatically (and so do layouts):

In [ ]:
auto_1 = pn.panel("# This becomes a Markdown pane")
auto_2 = {"a": 1, "b": 2}

pn.Column(auto_1, auto_2)

Use explicit pane types when you need predictable rendering options (and for efficiency).

## 3) Widgets: collect user input

Widgets are interactive controls with a `value` parameter.


In [ ]:
import panel_material_ui as pmui

name = pmui.TextInput(label="Name", placeholder="Type your name")
age = pmui.IntSlider(label="Age", start=0, end=100, value=30)

def render_preview(name, age):
    return f"Hello **{name or 'friend'}**, age **{age}**."

preview = pn.pane.Markdown(pn.bind(render_preview, name, age))

pn.Column(name, age, preview)

:::{tip}
Many widgets also expose throttled/input variants (like `value_throttled` or `value_input`) for better performance control.
:::

## 4) Layouts: arrange components

Layouts compose panes and widgets into an interface.

In [ ]:
filters = pn.Column(
    "### Filters",
    pmui.Select(name="Region", options=["All", "NA", "EU", "APAC"]),
    pmui.Checkbox(name="Show trend", value=True)
)

content = pn.Column(
    pn.pane.Markdown("### Dashboard"),
    pn.pane.Str("Put plots and tables here."),
)

pn.Row(filters, content, height=300)

List-like layouts (`Row`, `Column`, `Tabs`, `Accordion`, ...) support dynamic editing:

In [ ]:
col = pn.Column("### Log")
col.append("Started app")
col.append("Loaded data")
col

## (5) Shared *Parameters*

Widgets, indicators, panes and layouts all share a set of *Parameters*. Apart from those that are specific to layout and styling, which we will cover in the next sections) there's a few common ones. 

#### `loading`

Loading spinners are everywhere on modern apps and for good reasons, they let your users know that your app is doing some work for them! Fortunately they are very easy to set up in Panel, just set the `loading` *Parameter* to `True` or `False` on a component to display and hide its loading spinner.

In [ ]:
p_md = pmui.Column('# Title', loading=True)
p_md

In [ ]:
p_md.loading = False

#### `visible`

Sometimes it's useful to just completely hide a component, for instance to hide some advanced options. Again that's very easy to do, just set the `visible` *Parameter* to `False` on a component you want to hide.

In [ ]:
w_text = pmui.TextInput(label='Advanced', visible=False)
w_text

In [ ]:
w_text.visible = True

## 5) App shell: `Template` -> `panel_material_ui.Page`

Historically, many Panel apps started from a template object. In this codebase, prefer `panel_material_ui.Page` as the main app shell.

In [ ]:
import panel_material_ui as pmui

controls = pn.Column(
    pmui.Select(label="Region", options=["All", "NA", "EU"]),
    pmui.FloatSlider(label="Threshold", start=0, end=1, step=0.05, value=0.5),
)

main = pn.Column(
    "## KPI Overview",
    pn.pane.Markdown("Main content area"),
)

page = pmui.Page(
    title="Sales Monitor",
    sidebar=[controls],
    main=[main],
    sidebar_variant="persistent",
    sidebar_width=320,
)

page.preview()

Why this is the default direction:

- unified app-shell API (`header`, `sidebar`, `main`)
- first-class Material UI styling and theming
- clean path from prototype to polished app


## 6) Quick decision guide

- Use a **pane** when you already have output to render.
- Use a **widget** when you need user input.
- Use a **layout** when you need composition and structure.
- Use `panel_material_ui.Page` when you need a full app frame.


### Mini Exercise 1

Build a tiny app with:

1. one pane (`Markdown` or `DataFrame`)
2. two widgets (`Select` + `Slider`)
3. one layout (`Row` or `Column`)
4. wrapped in `pmui.Page`

Then explain in one sentence why each component is a pane, widget, layout, or page.

## Recap

The component hierarchy is simple once separated by intent:

- panes **display**
- widgets **input**
- layouts **compose**
- `panel_material_ui.Page` **frames the application**

That distinction makes your apps easier to design, debug, and scale.